# SmartSim tutorial 1:  getting started
In this notebook, we will walk through the most basic functionalities of SmartSim, such as setting up an experiment running two models, launching it locally, and collecting its results. We will also look at how we can use the `Ensemble` API to run models collectively. 

## 1.1 Running simple models 
The most common way of defining a workflow in SmartSim is through `Experiment`s. An experiment can start and stop a `Model` and check (`poll`) its status at any time. In section *1.2* we will also see how an `Experiment` can be used to run experiments as `Ensemble`s.

We begin by importing the modules we need: `Experiment` and `RunSettings`. `RunSettings` is the object used in SmartSim to define what will be run by a given `Model`. `RunSettings` is the most basic way of defining execution parameters, and will be perfect for executing programs launched locally, i.e. directly by the operating system, without a workload manager. We also import `os`, as we will need to setup the directory where the `Model`s will place their output and error files.

In [1]:
import os
from smartsim import Experiment
from smartsim.settings import RunSettings

Throughout this notebook, we will incrementally build an `Experiment`. Let's start from the simplest case: a single-`Model` example. Our first `Model` will simply print `hello`, using the shell command `echo`.

In [2]:
exp = Experiment(name="tutorial-experiment", launcher="local")

settings_1 = RunSettings(exe="echo", exe_args="hello")
M1 = exp.create_model(name="tutorial-model-1", run_settings=settings_1)

Once the `Model` has been created by the `Experiment`, we can start it. By setting `summary=True`, we can see a summary of the experiment printed before it is effectively launched. The summary will stay for 10 seconds, and it is useful as a last check. If we set `summary=False`, then the experiment would be launched immediately. We also explicitly set `block=True` (even though it is the default), so that  `Experiment.start` waits until the last `Model` has finished before returning: it will act like a job monitor, letting us know if processes run, complete, or fail.

In [3]:
exp.start(M1, block=True, summary=True)



=== LAUNCH SUMMARY ===
Experiment: tutorial-experiment
Experiment Path: /Users/arigazzi/Documents/DeepLearning/smartsim-dev/SmartSim/tutorials/01_getting_started/tutorial-experiment
Launching with: local
# of Ensembles: 0
# of Models: 1
Database: no

=== MODELS ===
tutorial-model-1
Model Parameters: 
{}
Model Run Settings: 
Executable: /bin/echo
Executable arguments: ['hello']







03:40:50 C02YR4ANLVCJ SmartSim[6230] INFO tutorial-model-1(6241): Completed


The model has completed. Let's look at the content of the current working directory.

In [4]:
os.listdir('.')

outputfile = './tutorial-model-1.out'
errorfile = './tutorial-model-1.err'

print("Content of tutorial-model-1.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-1.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

Content of tutorial-model-1.out:
hello

Content of tutorial-model-1.err:



We can see that two files, `tutorial-model-1.out` and `tutorial-model-1.err` have been created. The `.out` file contains the output generated by `model-1`, and the `.err` file would contain the error messages generated by it. Since there were no errors, the `.err` file is empty.

Now let's run two different `Model` instances at the same time. This is just as easy as running one `Model`, and takes the same steps. This time, we will skip the summary. For each `Model`, we create a `RunSettings` object: it is recommended to always create separate `RunSettings` objects for each `Model`.

In [5]:
exp = Experiment(name="tutorial-experiment", launcher="local")

run_settings_1 = RunSettings("sleep", "3")
run_settings_2 = RunSettings("sleep", "5")
model_1 = exp.create_model("tutorial-model-1", run_settings_1)
model_2 = exp.create_model("tutorial-model-2", run_settings_2)
exp.start(model_1, model_2)

03:40:57 C02YR4ANLVCJ SmartSim[6230] INFO tutorial-model-1(6243): Completed
03:40:57 C02YR4ANLVCJ SmartSim[6230] INFO tutorial-model-2(6244): Running
03:40:58 C02YR4ANLVCJ SmartSim[6230] INFO tutorial-model-1(6243): Completed
03:40:58 C02YR4ANLVCJ SmartSim[6230] INFO tutorial-model-2(6244): Completed


Again, we can check the content of the output and error files.

In [6]:
outputfile = './tutorial-model-1.out'
errorfile = './tutorial-model-1.err'

print("Content of tutorial-model-1.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-1.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

outputfile = './tutorial-model-2.out'
errorfile = './tutorial-model-2.err'

print("Content of tutorial-model-2.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-2.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

Content of tutorial-model-1.out:

Content of tutorial-model-1.err:

Content of tutorial-model-2.out:

Content of tutorial-model-2.err:



In many cases, a launcher different from `local` can be useful. For example, if `mpirun` is installed on the system, we can run a model through it, by specifying it as `run_command` in `RunSettings`. Since `mpirun` takes arguments (e.g. to define how many processes will be run), we pass them by defining `run_args` in `RunSettings`.

In [7]:
exp = Experiment("tutorial", launcher="local")
run_settings = RunSettings("echo",
                           "hello world!",
                           run_command="mpirun",
                           run_args={"-np": 2}) # note that for base ``RunSettings`` run_args passed literally
                      
model = exp.create_model("tutorial-model-mpirun", run_settings)
exp.start(model, summary=True)



=== LAUNCH SUMMARY ===
Experiment: tutorial
Experiment Path: /Users/arigazzi/Documents/DeepLearning/smartsim-dev/SmartSim/tutorials/01_getting_started/tutorial
Launching with: local
# of Ensembles: 0
# of Models: 1
Database: no

=== MODELS ===
tutorial-model-mpirun
Model Parameters: 
{}
Model Run Settings: 
Executable: /bin/echo
Executable arguments: ['hello', 'world!']
Run Command: mpirun
Run arguments: {'-np': 2}






03:41:16 C02YR4ANLVCJ SmartSim[6230] INFO tutorial-model-mpirun(6283): Completed


This time, since we passed `-np 2` to `mpirun`, in the output file we should find the line `hello world!` twice.

In [8]:
outputfile = './tutorial-model-mpirun.out'
errorfile = './tutorial-model-mpirun.err'

print("Content of tutorial-model-mpirun.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-mpirun.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

Content of tutorial-model-mpirun.out:
hello world!
hello world!
hello world!
hello world!
hello world!
hello world!
hello world!
hello world!

Content of tutorial-model-mpirun.err:



## 1.2 Creating and running replicas and ensembles of models
In the previous example, the two `Model` instances were created separately. There are more convenient ways of doing this, through `Ensemble`s. The first way we are going to see concerns running the same exact model several times. We first set up the example the standard way.

In [11]:
exp = Experiment(name="tutorial-experiment", launcher="local")
settings_1 = RunSettings(exe="sleep", exe_args="3")


Then, instead of creating it as we did before, we use `create_ensemble`. Let's assume we want to run the same experiment four times, then we will pass the `replicas=4` argument and simply start the `Ensemble`.

In [12]:
ensemble = exp.create_ensemble("ensemble-replica", replicas=4, run_settings=settings_1)
exp.start(ensemble, summary=True)



=== LAUNCH SUMMARY ===
Experiment: tutorial-experiment
Experiment Path: /Users/arigazzi/Documents/DeepLearning/smartsim-dev/SmartSim/tutorials/01_getting_started/tutorial-experiment
Launching with: local
# of Ensembles: 1
# of Models: 0
Database: no

=== ENSEMBLES ===
ensemble-replica
# of models in ensemble: 4
Launching as batch: False
Run Settings: 
Executable: /bin/sleep
Executable arguments: ['3']







03:42:43 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_0(6305): Completed
03:42:43 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_1(6306): Completed
03:42:43 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_2(6307): Completed
03:42:43 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_3(6308): Completed
03:42:44 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_0(6305): Completed
03:42:44 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_1(6306): Completed
03:42:44 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_2(6307): Completed
03:42:47 C02YR4ANLVCJ SmartSim[6230] INFO ensemble-replica_3(6308): Completed


From the output, we see that four copies of our `Model`, named `ensemble-replica_0`, `ensemble-replica_1`, ... were run. In each output file, we will see that the same output was generated.

Now let's imagine that we don't want to run the *same* model four times, but we want to run variations of it. One way of doing this would be to define four models, and starting them through the `Experiment`.
For few, simple `Model`s, this woukd OK, but what if we needed to run a large number of models, which only differ for some parameter? Defining and adding each one separately would be tedious. For such cases, we will rely on a parameterized `Ensemble` of models.

Our goal is to run 

```python output_my_parameter.py```

 changing some internal parameters of `output_my_parameter.py`. Clearly, we could pass the parameters as arguments, but in some cases, this could not be possible (e.g. if the parameters were stored in a file and the executable would not accept them from the command line). We begin by defining the `Experiment` in the standard way.

In [13]:
exp = Experiment("tutorial-ensemble", launcher="local")
rs = RunSettings(exe="python", exe_args="output_my_parameter.py")

Then, we define the parameters we are going to set: `tutorial_name` and `tutorial_parameter`. In the original file `output_my_parameter.py`, which acts as a template, they occur as `;tutorial_name;` and `;tutorial_parameter;`. The semi-colons are used to perform a regexp substitution with the desired values. We pass them to `create_ensemble`, along with the argument `perm_strategy="all_perm"`. This argument means that we want all possible permutations of the given parameters, which are stored in the dict `params`. We have two options for both paramters, thus our ensemble will run 4 instances of the same `Experiment`, just using a different copy of `output_my_parameter.py`. We attach the template file to the `Ensemble` instance and we run the experiment.

In [14]:
params = {"tutorial_name": ["Ellie", "John"], "tutorial_parameter": [2, 11]}
ensemble = exp.create_ensemble("ensemble", params=params, run_settings=rs, perm_strategy="all_perm")
config_file = "./output_my_parameter.py"
ensemble.attach_generator_files(to_configure=config_file)

exp.generate(ensemble, overwrite=True)
exp.start(ensemble)

03:43:08 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_0(6315): Completed
03:43:08 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_1(6316): Completed
03:43:08 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_2(6317): Completed
03:43:08 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_3(6318): Completed
03:43:09 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_0(6315): Completed
03:43:09 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_1(6316): Completed
03:43:09 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_2(6317): Completed
03:43:12 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_3(6318): Completed


We can see from the output that four instances of our experiment were run, each one named like the `Experiment`, with a numeric suffix at the end: `ensemble_0`, `ensemble_1`, ... each ensemble member generated its own output files, which will be stored in `tutorial-ensemble/ensemble/ensemble_0`, `tutorial-ensemble/ensemble/ensemble_1`, and so on.

In [15]:
for ensemble_id in range(4):
    outputfile = 'tutorial-ensemble/ensemble/ensemble_' + str(ensemble_id)+"/ensemble_"+ str(ensemble_id)+".out"

    print(f"Content of {outputfile}:")
    with open(outputfile, 'r') as fin:
        print(fin.read())


Content of tutorial-ensemble/ensemble/ensemble_0/ensemble_0.out:
Hello, my name is Ellie and my parameter is 2

Content of tutorial-ensemble/ensemble/ensemble_1/ensemble_1.out:
Hello, my name is Ellie and my parameter is 11

Content of tutorial-ensemble/ensemble/ensemble_2/ensemble_2.out:
Hello, my name is John and my parameter is 2

Content of tutorial-ensemble/ensemble/ensemble_3/ensemble_3.out:
Hello, my name is John and my parameter is 11



That's it! All possible permutations of the input parameters were used to execute the experiment! Sometimes, the parameter space can be too large to be explored exhaustively. In that case, we can use a different permutation strategy, i.e. `random`. For example, if we want to only use two possible random combinations of our parameter space, we can run the following code, where we specift `n_models=2` and `perm_strategy="random"`.

In [16]:
params = {"tutorial_name": ["Ellie", "John"], "tutorial_parameter": [2, 11]}
ensemble = exp.create_ensemble("ensemble", params=params, run_settings=rs, perm_strategy="random", n_models=2)
config_file = "./output_my_parameter.py"
ensemble.attach_generator_files(to_configure=config_file)

exp.generate(ensemble, overwrite=True)
exp.start(ensemble)

03:43:13 C02YR4ANLVCJ SmartSim[6230] INFO Working in previously created experiment
03:43:18 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_0(6327): Completed
03:43:18 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_1(6328): Completed
03:43:19 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_0(6327): Completed
03:43:19 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_1(6328): Completed


Another possible permutation strategy is `stepped`, but it is also possible to pass a function, which will need to generate combinations of parameters starting from the dictionary. Please refer to the documentation to learn more about this.


It is also possible to use different delimiters for the parameter regexp. For example, if instead of `;`, we want to use `@`, we can set it as `tag` in `generate`. We have to use a different version of the parameterized file, one named `output_my_parameter_new_tag.py`.

In [17]:
exp = Experiment("tutorial-ensemble-new-tag", launcher="local")
rs = RunSettings(exe="python", exe_args="output_my_parameter_new_tag.py")
params = {"tutorial_name": ["Ellie", "John"], "tutorial_parameter": [2, 11]}
ensemble = exp.create_ensemble("ensemble", params=params, run_settings=rs, perm_strategy="all_perm")
config_file = "./output_my_parameter_new_tag.py"
ensemble.attach_generator_files(to_configure=config_file)

exp.generate(ensemble, overwrite=True, tag='@')
exp.start(ensemble)

03:43:29 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_0(6329): Completed
03:43:29 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_1(6330): Completed
03:43:29 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_2(6331): Completed
03:43:29 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_3(6332): Completed
03:43:30 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_0(6329): Completed
03:43:30 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_1(6330): Completed
03:43:30 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_2(6331): Completed
03:43:33 C02YR4ANLVCJ SmartSim[6230] INFO ensemble_3(6332): Completed


## 1.3 Launching and communicating with the orchestrator

In this section we will see how to use the `Orchestrator` and `SmartRedis` to interact with an in-memory DB. We start by importing the SmartRedis `Client` and the `Orchestrator`

In [18]:
from smartredis import Client
from smartsim.database import Orchestrator
import numpy as np

REDIS_PORT=6899

As first thing, we will start the `Orchestrator`. Since we are setting `launcher="local"` in `Experiment`, the `Orchestrator` will run a single DB instance, once we start the `Experiment`.

In [19]:

exp = Experiment("tutorial-smartredis", launcher="local")

# create and start a database
orc = Orchestrator(port=REDIS_PORT)
exp.generate(orc)
exp.start(orc, block=False)

Now that the `Orchestrator` is running, we can use SmartRedis to store NumPy tensors on the Redis DB, and get them back. This is done using the SmartSim `Client`. First, we setup a connection to the DB.

In [20]:
client = Client(address='127.0.0.1:'+str(REDIS_PORT), cluster=False)

Then, we can use the DB to put and retrieve tensors. We need to assign a unique key to each tensor (or object) we store on the DB.

In [22]:
send_tensor = np.ones((4,3,3))

client.put_tensor("tutorial_tensor_1", send_tensor)

receive_tensor = client.get_tensor("tutorial_tensor_1")

print('Receive tensor:\n\n', receive_tensor)

Receive tensor:

 [[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


Thanks to the SmartRedis `Client` and its possibility to access functions from the RedisAI module, we can store and run a NN model directly on the DB node. We first create a one-layer PyTorch Convolutional Neural Network, and save it as a jit-traced, serialized, object.

In [23]:
import torch
import torch.nn as nn

# taken from https://pytorch.org/docs/master/generated/torch.jit.trace.html
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1, 1, 3)

    def forward(self, x):
        return self.conv(x)


net = Net()
example_forward_input = torch.rand(1, 1, 3, 3)
module = torch.jit.trace(net, example_forward_input)

# Save the traced model to a file
torch.jit.save(module, "./torch_cnn.pt")   

Now we send the model to the database, again, we assign it a unique key, `tutorial-cnn`, which we will use to refer to the model when using the `Client`.

In [24]:
# Set the model in the Redis database from the file
client.set_model_from_file("tutorial-cnn", "./torch_cnn.pt", "TORCH", "CPU")

Now we create a random tensor, store it on the DB, and use it as input to the CNN we just sent. RedisAI will run the model and store the output with the key we specify. Using that key, we can retrieve the tensor.

In [25]:
# Put a tensor in the database as a test input
data = torch.rand(1, 1, 3, 3).numpy()
client.put_tensor("torch_cnn_input", data)

# Run model and retrieve the output
client.run_model("tutorial-cnn", inputs=["torch_cnn_input"], outputs=["torch_cnn_output"])
out_data = client.get_tensor("torch_cnn_output")

Notice that we could have defined the model as an object (without storing it on disk) and send it to the DB using `set_model` instead of `set_model_from_file`. We can do the same thing for any Python function. For example, let's define a simple function takes a NumPy tensor as input.

In [26]:
def max_of_tensor(array):
    """Sample torchscript script that returns the
    highest element in an array.

    """
    # return the highest element
    return array.max(1)[0]

sample_array_1 = np.array([np.arange(9.)])
print(sample_array_1)
print("Max:")
print(max_of_tensor(sample_array_1))

[[0. 1. 2. 3. 4. 5. 6. 7. 8.]]
Max:
8.0


Now let's store this function on the DB, assignign it the key `max-of-tensor`: 

In [27]:
client.set_function("max-of-tensor", max_of_tensor)

Now we perform the same sample computation on the DB. 

In [29]:
client.put_tensor("script-data-1", sample_array_1)
client.run_script(
    "max-of-tensor",  # key of our script
    "max_of_tensor",  # function to be called
    ["script-data-1"],
    ["script-output"],
)

out = client.get_tensor("script-output")

print(out)

[8.]


And, as expected, we obtain the same result we obtained when we ran the function locally. To clean up, we need to tear down the DB. We do this by stopping the `Orchestrator`.

In [30]:
exp.stop(orc)

03:44:44 C02YR4ANLVCJ SmartSim[6230] INFO Stopping model orchestrator_0 with job name orchestrator_0-CACUV0TN47RK


## 1.4 Ensembles using SmartRedis

In Section 1.2 we have seen `Ensemble`s. What would happen if `Model`s which are part of an `Ensemble` tried to put their tensors on the DB using SmartRedis? Unless we used unique keys across the running programs, several tensors (or objects) would have the same key, and this key collision would result in unexpected behavior. In other words, if in the source code of one program, a tensor with key `tensor1` was put on the DB, then each replica of the program would put a tensor with the key `tensor1`. SmartSim and SmartRedis can avoid key collision by prepending program-unique prefixes to entities. 

Let's start by setting up the experiment with the `Orchestrator`.

In [31]:
exp = Experiment("tutorial-smartredis-ensemble", launcher="local")

# create and start a database
orc = Orchestrator(port=REDIS_PORT)
exp.generate(orc)
exp.start(orc, block=False)

03:44:44 C02YR4ANLVCJ SmartSim[6230] INFO Working in previously created experiment


Now let's add two replicas of the same `Model`. Basically, it is a simple producer, which puts a tensor on the DB. The code for it is in `producer.py`.

In [32]:
rs_prod = RunSettings("python", "producer.py --redis-port "+str(REDIS_PORT))
ensemble = exp.create_ensemble(name="producer",
                               replicas=2, 
                               run_settings=rs_prod)

We add a consumer, which will just retrieve the tensors put by the two producers and check that they are what it expects.

In [33]:
rs_consumer = RunSettings("python", "consumer.py --redis-port "+str(REDIS_PORT))
consumer = exp.create_model("consumer", run_settings=rs_consumer)

We need to register incoming entities, i.e. entities for which the prefix will have to be known by other entities. When we will start the `Experiment`, environment variables will be set to let all entities know which incoming entities are present.

In [34]:
consumer.register_incoming_entity(ensemble[0])
consumer.register_incoming_entity(ensemble[1])

Finally, we attach the files to the experiments, generate them, and run!

In [35]:
ensemble.attach_generator_files(to_copy=['producer.py'])
consumer.attach_generator_files(to_copy=['consumer.py'])
exp.generate(ensemble, overwrite=True)
exp.generate(consumer, overwrite=True)

# start the models
exp.start(ensemble, consumer, summary=True)

03:45:15 C02YR4ANLVCJ SmartSim[6230] INFO Working in previously created experiment
03:45:15 C02YR4ANLVCJ SmartSim[6230] INFO Working in previously created experiment


=== LAUNCH SUMMARY ===
Experiment: tutorial-smartredis-ensemble
Experiment Path: /Users/arigazzi/Documents/DeepLearning/smartsim-dev/SmartSim/tutorials/01_getting_started/tutorial-smartredis-ensemble
Launching with: local
# of Ensembles: 1
# of Models: 1
Database: no

=== ENSEMBLES ===
producer
# of models in ensemble: 2
Launching as batch: False
Run Settings: 
Executable: /usr/local/anaconda3/envs/smartsim/bin/python
Executable arguments: ['producer.py', '--redis-port', '6899']



=== MODELS ===
consumer
Model Parameters: 
{}
Model Run Settings: 
Executable: /usr/local/anaconda3/envs/smartsim/bin/python
Executable arguments: ['consumer.py', '--redis-port', '6899']







03:45:29 C02YR4ANLVCJ SmartSim[6230] INFO producer_0(6428): Completed
03:45:29 C02YR4ANLVCJ SmartSim[6230] INFO consumer(6430): Completed
03:45:30 C02YR4ANLVCJ SmartSim[6230] INFO producer_1(6429): Completed
03:45:33 C02YR4ANLVCJ SmartSim[6230] INFO producer_1(6429): Completed


The producers produced random NumPy tensors, and we can see that the consumer was able to retrieve both of them from the DB, by looking at its output.

In [36]:
outputfile = './tutorial-smartredis-ensemble/consumer/consumer.out'

with open(outputfile, 'r') as fin:
    print(fin.read())

Tensor for producer_0 is: [[[[0.58531817 0.30393151 0.28384912]
   [0.38601273 0.88431641 0.08462409]
   [0.91530914 0.95270186 0.77886599]]]]
Tensor for producer_1 is: [[[[0.56121212 0.07056607 0.25703332]
   [0.03478823 0.69236882 0.99710588]
   [0.03601962 0.12958589 0.08349446]]]]



As usual, let's shutdown the DB, by stopping the `Orchestrator`.

In [37]:
exp.stop(orc)

03:45:35 C02YR4ANLVCJ SmartSim[6230] INFO Stopping model orchestrator_0 with job name orchestrator_0-CACUVX62J3Y8
